In [1]:
import polars as pl
import json
from pprint import pprint
pl.Config.set_tbl_rows(50)



polars.config.Config

In [2]:
ITENS_COLUNAS = [
    'item',
    'un',
    'qt',
    'valor_unidade',
    'valor_unidade_total'
]

UNKNOWN_TEXT = 'Desconhecido'

# Dados para visualização
VIS_COLS = [
    'id_nota_fiscal',
    'valor_nota_fiscal',
    'item_deduplicado',
    'qt',
    'valor_unidade',
    'valor_unidade_total',
]


In [19]:
df = pl.read_parquet('data/banco_analise_2.parquet')

df_enriched = df.filter(pl.col('quantidade_item').is_not_null())

item_description = json.load(open('item_cache.json', 'r'))


In [20]:
total = len(df)

print(f"Número de linhas: {total:,}")

q1 = len(df.filter(
    (pl.col('valor_unidade') == 0) & 
    (pl.col('valor_unidade_total') != 0) & 
    (pl.col('qt') != 0)
))

print(
    f"Registros com valor_unidade = 0 e valor_unidade_total != 0: "
    f"{q1:,} ({q1/total:.2%})"
)

q2 = len(df.filter((pl.col('valor_unidade') == 0) & (pl.col('valor_unidade_total') == 0)))
print(
    f"Registros com valor_unidade = 0 e valor_unidade_total = 0: "
    f"{q2:,} ({q2/total:.2%})"
)

print(
    'Número de items extraídos', len(item_description.keys())
)

Número de linhas: 64,703,191
Registros com valor_unidade = 0 e valor_unidade_total != 0: 23,392,240 (36.15%)
Registros com valor_unidade = 0 e valor_unidade_total = 0: 2,151 (0.00%)
Número de items extraídos 104185


In [5]:
unidades_corrigidas = {
    "UNID": 'UN', 
    "UN": 'UN',
    "QUILOGRAMA": 'KG',
    "KG": 'KG',
    "QUILOGRAMAS": 'KG',
    "LITRO": 'L'
}   

df_enriched = df_enriched.with_columns(
    pl.col("unidade_item")
    .replace(unidades_corrigidas, default=pl.col("unidade_item"))
    .alias("unidade_item")
)

/var/folders/42/_z7t3f3n2b74bggwwctg77z80000gn/T/ipykernel_64555/110374142.py:12: DeprecationWarning: the `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
(Deprecated in version 1.0.0)
  .replace(unidades_corrigidas, default=pl.col("unidade_item"))


In [14]:
df_enriched['unidade_item'].value_counts().sort(by='count', descending=True)

unidade_item,count
str,u32
"""Desconhecido""",61526
"""G""",13132
"""KG""",11926
"""ML""",2204
"""GR""",2055
"""PCT""",1164
"""UN""",1023
"""L""",360
"""CX""",324


In [7]:
df_enriched.filter(pl.col('unidade_item') == 'UN')[['item', 'item_deduplicado', 'valor_unidade', 'unidade_item', 'un']]

item,item_deduplicado,valor_unidade,unidade_item,un
str,str,f32,str,str
"""TEMPERO VERDE UN""","""TEMPERO VERDE""",0.9,"""UN""","""UNB"""
"""GOIABA VERMELHA TIPO 12-UN""","""GOIABA VERMELHA TIPO""",0.69,"""UN""","""UN"""
"""GOIABA VERMELHA TIPO 12-UN""","""GOIABA VERMELHA TIPO""",0.69,"""UN""","""UN"""
"""GOIABA VERMELHA TIPO 12-UN""","""GOIABA VERMELHA TIPO""",0.69,"""UN""","""UN"""
"""GOIABA VERMELHA TIPO 12-UN""","""GOIABA VERMELHA TIPO""",0.69,"""UN""","""UN"""
"""GOIABA VERMELHA TIPO 12-UN""","""GOIABA VERMELHA TIPO""",0.69,"""UN""","""UN"""
"""GOIABA VERMELHA TIPO 12-UN""","""GOIABA VERMELHA TIPO""",0.69,"""UN""","""UN"""
"""GOIABA VERMELHA TIPO 12-UN""","""GOIABA VERMELHA TIPO""",0.69,"""UN""","""UN"""
"""GOIABA VERMELHA TIPO 12-UN""","""GOIABA VERMELHA TIPO""",0.69,"""UN""","""UN"""


# Calculate percent values

In [8]:
(
    df_enriched.group_by(by=['item_deduplicado', 'ano'])
      .agg([
          pl.col("valor").quantile(0.25).alias("p25"),
          pl.col("valor").quantile(0.50).alias("p50"),  # mediana
          pl.col("valor").quantile(0.75).alias("p75"),
      ])
)

TypeError: Expected Polars expression or object convertible to one, got <class 'list'>.

Hint: if you tried
    group_by(by=['item_deduplicado', 'ano'])
then you probably want to use this instead:
    group_by(['item_deduplicado', 'ano'])

Número de linhas: 64,703,191
Valor unitário igual a 0: 23,394,729
Valor unitário total igual a 0: 6,214
